In [1]:
from joblib import Parallel, delayed
from sim.dy import Model
from sim import load_inputs
import json
import pandas as pd
import matplotlib.pylab as plt
import os

In [2]:
with open('data/test_priors.json', 'r') as f:
    prior = json.load(f)

print(f'Prior size: {len(prior)}')
    
inputs = load_inputs('data/pars.json')

m = Model(inputs, year0=1970)

Prior size: 200


In [10]:
def fn(i, p0, model):
    _, ms, msg = model.simulate(p0)
    if msg['succ']:
        ms = pd.DataFrame(ms).assign(Key = i)
        return ms
    else:
        return None

In [11]:
with Parallel(n_jobs=5, verbose=4) as parallel:
    mss = parallel(delayed(fn)(i, p, m) for i, p in enumerate(prior))

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    6.8s
[Parallel(n_jobs=5)]: Done  88 tasks      | elapsed:   25.6s
[Parallel(n_jobs=5)]: Done 200 out of 200 | elapsed:   54.7s finished


In [5]:
mss = [ms for ms in mss if ms is not None]
mss = pd.concat(mss)
mss.head()

NameError: name 'mss' is not defined

In [ ]:
out_path = 'out/dy_free'

os.makedirs(out_path, exist_ok=True)

mss.to_csv(f'{out_path}/Runs_Prior.csv')